Deep contextualized word representation has drawn wide attention because of state-of-the-art performances in downstream tasks. Contextualized embeddings can capture not only word-level information but also multi-sense information, thus improving the results in sentiment analysis, SQuad and etc. However, the language adopted in the [Elmo](https://allennlp.org/elmo) model were biLSTMs which contained a huge number of parameters, it was less likely for small labs to train and run such experiments.


In this project, we intend to make use of CNN language model in learning efficient word representations for sentiment analysis. We train a language model based on [Gated CNN architecture](https://arxiv.org/abs/1612.08083) proposed by Yann Daulphin, then do sentiment analysis with embeddings generated by the language model.

The language model training dataset is 1-billion-word-language.

In [1]:
import tensorflow as tf
import os
import time

from model import *
from data_utils import data_helper
from conf_utils import *

/home/nlp/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Initialize the configuration and prepare data batches

In [2]:
class config:
    vocab_size = 2000
    embedding_size = 200
    filter_size = 64
    num_layers = 1
    block_size = 2
    filter_h = 5
    context_size = 20
    text_size = context_size
    batch_size = 16
    epochs = 5
    num_sampled = 64
    learning_rate = 0.0001
    momentum = 0.99
    grad_clip = 0.1
    num_batches = 0
    ckpt_path = 'ckpt'
    summary_path = 'logs'
    #data_dir = "data/texts/reviews/movie_reviews"
    data_dir = "data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled"

In [3]:
#Initialize configuration files
conf = prepare_conf(config)

In [4]:
#Create data batches for language model training
dh = data_helper(conf)
x_batches, y_batches, word_to_idx, idx_to_word = dh.prepare_data()

In [5]:
conf.text_size

22

In [6]:
y_batches[0][0]

array([  1,   2,  67,   0,   6,  57,   0, 611, 134, 650,   7,   0,  46,
        14,   9,   0, 427,  31, 785,   4,   3,   1])

## Train a CNN-based language model

In [7]:
#Create a language model
#Note we need to save the models for subsequent tasks
model = GatedCNN(conf)
saver = tf.train.Saver(tf.trainable_variables())
print("Started Model Training...")

Tensor("layer_0/mul:0", shape=(16, 22, 1, 64), dtype=float32)
Started Model Training...


In [20]:
#from RNN import RNN
#model = RNN(conf)
#saver = tf.train.Saver(tf.trainable_variables())

In [21]:
#labels.shape

In [22]:
model.y

<tf.Tensor 'Reshape_1:0' shape=(352, 1) dtype=int32>

In [8]:
batch_idx = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(conf.summary_path, graph=sess.graph)

    #if os.path.exists(conf.ckpt_file):
        #saver.restore(sess, conf.ckpt_file)
        #print("Model Restored")

    for i in np.arange(conf.epochs):
        start = time.time()
        for j in np.arange(conf.num_batches):
        #for j in np.arange(21):
            inputs, labels, batch_idx = dh.get_batch(x_batches, y_batches, batch_idx)
            _, l = sess.run([model.optimizer, model.loss], feed_dict={model.X:inputs, model.y:labels})
            if j%20 == 0:
                print(l)
        end = time.time()
        print("Epoch: %.2f, Time: %.2f,  Loss: %.2f"%(i, end-start, l))

        if i % 2 == 0:
            perp = sess.run(model.perplexity, feed_dict={model.X:inputs, model.y:labels})
            print("Perplexity: %.2f"%perp)
            saver.save(sess, conf.ckpt_file)

        summaries = sess.run(model.merged_summary_op, feed_dict={model.X:inputs, model.y:labels})
        summary_writer.add_summary(summaries, i)

223.26772
227.70192
227.99956
208.36089
212.4167
203.27255
224.46005
185.67218
177.58745
188.18663
146.32607
168.51358
150.96786
156.16171
145.6881
179.34212
152.08679
137.91362
97.152664
113.898895
121.605545
139.27708
119.64997
118.64231
134.82767
143.98682
100.1643
105.68909
121.312454
98.34222
81.95305
126.8116
83.44582
84.46463
93.157814
56.46525
43.83351
137.56476
117.19837
107.361275
57.000698
52.777164
97.905075
75.96489
53.79938
60.598644
80.37415
22.863533
83.861374
68.35846
65.68785
54.80743
42.50838
52.88417
74.56658
36.594654
41.575573
44.01676
47.03163
59.84403
53.362564
45.595596
46.375626
31.009205
14.648789
35.55819
44.863823
34.07427
53.857216
26.458221
43.773563
31.37784
51.65635
27.37025
19.509045
29.14902
66.71709
39.638447
21.53565
37.455814
21.847406
47.036217
37.34256
21.799898
23.319324
43.41062
39.995777
29.902403
22.831322
29.286768
14.550912
46.84936
24.484896
24.686182
37.054104
35.073746
21.852516
20.087107
40.469498
18.264572
12.286408
44.487442
54.399315

2.0339568
2.3527396
2.4298797
2.6817396
2.2533872
2.2600443
2.01495
2.234471
2.3969252
2.277185
2.3106058
2.3131597
2.4052653
2.5430412
2.205144
2.3562438
2.224881
2.3156374
2.1680062
2.117867
2.239049
2.07634
2.263456
2.4079225
2.548605
2.3143923
2.2672546
2.0523858
2.1438358
2.1201627
2.2864027
2.2327485
2.2195096
2.106459
2.387413
2.369789
2.1371005
2.2776346
2.1769857
2.5883331
2.165829
1.8974999
2.3811598
2.034783
2.2098422
2.361374
2.201789
2.3701613
2.4126127
2.6131358
2.2021182
2.2743728
2.285739
2.1355612
3.0090544
2.4865835
2.4310586
2.27762
2.1879964
2.2732286
2.4407802
2.3321545
2.255269
2.0162382
2.085239
2.3517885
2.210936
2.2794175
2.191223
2.0150473
1.9632041
2.1461573
2.1476204
2.421642
2.1116614
2.3050086
2.0598266
2.1250606
2.1899548
2.322623


KeyboardInterrupt: 

## Train a RNN model

## Sentiment Analysis

In this part, we need to use other datasets for sentiment analysis, like the one of SemEval2013.

In [1]:
#Use spacy tokenize sentences into words
#!pip install spacy
#!python -m spacy download en
#import spacy
#nlp = spacy.load('en')
#def sent_split(sent):
    #words = []
    #sent = nlp(sent.strip())
    #for w in sent:
        #words.append(w.text.lower())
    #return words

In [2]:
file_train = 'data/semeval/downloaded.tsv'
file_dev = 'data/semeval/dev_downloaded.tsv'
file_test = 'data/semeval/test.txt'
with open(file_train) as f:
    tweets_train = f.readlines()
with open(file_dev) as f:
    tweets_dev = f.readlines()
with open(file_test) as f:
    tweets_test = f.readlines()
    


#Filter empty tweets
def is_available(text):
    if 'Not Available' in text:
        return False
    if '\t"objective' in text:
        return False
    if '\t"neutral' in text:
        return False
    if '\tobjective' in text:
        return False
    if '\tneutral' in text:
        return False
    return True


In [3]:
tweets_train = list(filter(is_available, tweets_train))
tweets_dev = list(filter(is_available, tweets_dev))
tweets_test = list(filter(is_available, tweets_test))

In [4]:
tweets_train = [item.split('\t') for item in tweets_train]
tweets_dev = [item.split('\t') for item in tweets_dev]
tweets_test = [item.split('\t') for item in tweets_test]
_, _, y_train, text_train = list(zip(*tweets_train))
_, _, y_dev, text_dev = list(zip(*tweets_dev))
_, _, y_test, text_test = list(zip(*tweets_test))


In [5]:
text_train, y_train = list(text_train), list(y_train)
text_dev, y_dev = list(text_dev), list(y_dev)
text_test, y_test = list(text_test), list(y_test)
y_test = ['"' + item + '"' for item in y_test]

In [6]:
#Encode the labels to numbers
from sklearn import preprocessing
label_encode = preprocessing.LabelEncoder()  # 建立模型
y_train = label_encode.fit_transform(y_train)
y_dev = label_encode.transform(y_dev)
y_test = label_encode.transform(y_test)

In [14]:
from sents_handler import generate_samples
import numpy as np
train_gs = generate_samples(np.array(text_train), np.array(y_train), word_to_idx, False)
sent_vecs, sent_labels, lengths = train_gs.generate(16)

In [12]:
with tf.Session() as sess:
    saver.restore(sess, conf.ckpt_file)
    #Get the contextualized representation
    out_layer = sess.run(model.out_layer, feed_dict={model.X:inputs})
    
    #out_layer.resha

INFO:tensorflow:Restoring parameters from ckpt/vocab2000_embed200_filters64_batch16_layers5_block2_fdim5/model.ckpt


In [17]:
max_word_len = out_layer.shape[1]
from sentiment_analysis import CNN_Model_Pretrained_Emb
class trainConfig:
    max_doc_len = max_word_len
    label_size = 2
    embed_size = 100
    hidden_size = 250
    batch_size = 64
    layer_size = 2
    
class testConfig:
    max_doc_len = max_word_len
    label_size = 2
    embed_size = 100
    hidden_size = 250
    batch_size = 64
    layer_size = 2
    
class singleConfig:
    max_doc_len = max_word_len
    label_size = 2
    embed_size = 100
    hidden_size = 250#hidden size for hidden state of rnn
    batch_size = 1
    layer_size = 2


In [18]:
import tensorflow as tf
graph_cnn = tf.Graph()
#Create models for training and testing data
with graph_cnn.as_default():
    initializer = tf.random_uniform_initializer(-0.02, 0.02)
    with tf.name_scope('train'):
        #Set different models for different buckets
        with tf.variable_scope("Model", reuse=None, initializer=initializer):
            train_model = CNN_Model_Pretrained_Emb(trainConfig)
            saver=tf.train.Saver()
    with tf.name_scope('test'):
        #Set different models for different buckets
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            test_model = CNN_Model_Pretrained_Emb(testConfig, False)
            single_model = CNN_Model_Pretrained_Emb(singleConfig, False)



Model Initialized!
Model Initialized!
Model Initialized!


In [ ]:
import time, os
epochs = 2
#train_chunk_num = 10
file = "ckpt_cnn_pretrained_emb/cnn.ckpt"
with tf.Session(graph=graph_cnn) as sess:
    #Initialize parameters
    init = tf.global_variables_initializer()
    if not os.path.exists("ckpt_cnn_pretrained_emb"):
        os.mkdir('ckpt_cnn_pretrained_emb')
    if os.path.exists("ckpt_cnn_pretrained_emb/cnn.ckpt.index"):
        saver.restore(sess, file)
    else:
        sess.run(init)
    start_time = time.time()
    for m in range(epochs):
        for i in range(train_chunk_num):
            #sess.run(tf.assign(learning_rate, 0.002*((0.98)**m)))
            x, y, lengths = train_gs.generate(trainConfig.batch_size)
            feed_dict = {train_model.x:x, train_model.y:y, train_model.lengths:lengths}
            l, _ = sess.run([train_model.cost, train_model.optimize], feed_dict=feed_dict)
            if i%100 == 0:
                print('Loss:', round(l, 4))
        end_time = time.time()
        print('Epoch', m, 'time:{:.2f}'.format(end_time - start_time))
        
    saver.save(sess,'ckpt_cnn_pretrained_emb/cnn.ckpt')

In [ ]:
#Calculate Testing Accuracy
with tf.Session(graph=graph_cnn) as sess:
    print('Testing...')
    count = 0
    #saver = tf.train.import_meta_graph('ckpt_cnn/cnn.ckpt.meta')
    saver.restore(sess,tf.train.latest_checkpoint('ckpt_cnn_pretrained_emb/'))
    print('Parameters restored')
    start_time = time.time()
    test_gs = generate_samples(np.array(test_processed), np.array(test_labels), False)
    for _ in range(test_chunk_num):
        #Traverse each data
        x, y, lengths = test_gs.generate(testConfig.batch_size)
        feed_dict = {test_model.x:x, test_model.y:y, test_model.lengths:lengths}
        n = sess.run(test_model.correct_num, feed_dict=feed_dict)
        count += np.sum(n)
    for _ in range(remain_num):
        #Traverse each data
        x, y, lengths = test_gs.generate(1)
        feed_dict = {single_model.x:x, single_model.y:y, 
                     single_model.lengths:lengths}
        n = sess.run(single_model.correct_num, feed_dict=feed_dict)
        count += np.sum(n)
    end_time = time.time()
    print('Testing Time:{:.2f}'.format(end_time - start_time))
    print(count*1.0/len(test_processed))